In [25]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.cm as cm

import os
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import OneClassSVM

In [42]:
!wc -l swab_seqs/*

   637811 swab_seqs/SRS015895seq.csv
   652089 swab_seqs/SRS017537seq.csv
   139795 swab_seqs/SRS018661seq.csv
   380992 swab_seqs/SRS019024seq.csv
   632243 swab_seqs/SRS019329seq.csv
  2442930 total


In [13]:
!head -n 1 swab_seqs/SRS015895seq.csv

class,gen_seq,unique_id


In [14]:
!head -n 1 c4_bacterial_genome.csv

g_s,genome,tag


In [41]:
full = pd.DataFrame(columns=['CLASS','GEN_SEQ'])
swab_base = './swab_seqs/'
for file in os.listdir(swab_base):
    i = 0
    for df in pd.read_csv(swab_base+file, chunksize=10**5):
        size = len(df)
        full = full.append(pd.DataFrame({'CLASS':['unknown']*size, 'GEN_SEQ': df.gen_seq.values}))

print(len(full))
for df in pd.read_csv('c4_bacterial_genome.csv', chunksize=10**5):
    size = len(df)
    full = full.append(pd.DataFrame({'CLASS':['bacteria']*size, 'GEN_SEQ': df.genome.values}))
print(len(full))
full = full.reset_index(drop=True)
full.head()

2442925
2443268


,CLASS,GEN_SEQ
0,unknown,AAGACGCATCAAACATAGCAAAACGTTTTGCATTCTCAACCGCACT...
1,unknown,AAATTATGCATATTACGCTTTGCATAAATTTAGGTGGACACCATCC...
2,unknown,GCTGCTTTATCAGCGAAACGCTGACTTATGCCATCAACGAGCTGAA...
3,unknown,CACGCCCCTTGCCGTGCAGGGGGTAGGGACGAGCTGATAAGTACTC...
4,unknown,GTATCGGGAATGTCGTGATCGCACCAAATACGAAATACCTCACTGA...


In [44]:
vec = TfidfVectorizer(analyzer='char', ngram_range=(5,5))
%time vec_X = vec.fit_transform(full.GEN_SEQ.values)

CPU times: user 6min 6s, sys: 14.6 s, total: 6min 20s
Wall time: 6min 20s


## Train [Novelty Classifier](http://scikit-learn.org/stable/modules/outlier_detection.html) on Reference Bacteria Data for it to treat outliers as human samples

In [62]:
novelty = OneClassSVM(kernel='rbf',)
bac_idx = full[full.CLASS == 'bacteria'].index.values
%time novelty.fit(vec_X[bac_idx])

CPU times: user 96 ms, sys: 0 ns, total: 96 ms
Wall time: 95.2 ms


OneClassSVM(cache_size=200, coef0=0.0, degree=3, gamma='auto', kernel='rbf',
      max_iter=-1, nu=0.5, random_state=None, shrinking=True, tol=0.001,
      verbose=False)

In [75]:
preds = novelty.predict(vec_X)

In [81]:
len(preds[preds > 0])

82

## 82 out of the 2443186 samples from full dataset (which includes the training set) were classified as Bacteria.

### 99.997% Human vs 0.003% Bacterial


## Conclusion: We require much more model tuning before reporting any numbers.
